open X:\Macropinocytosis\BMM-KOs\20230823cs5cell1WT\*488nm*.tif;

#surface render#

material dull;

lighting full;

lighting depthcue false;

set bg transparent;

windowsize 800 800;

volume #1 step 3 level 145 color #023b55 voxelsize 0.1028;

view orient;

ui tool show Shell

In [ ]:
from chimerax.color_key import show_key
from chimerax.core import colors
from chimerax.std_commands.wait import wait
from chimerax.surface import vertex_convexity
from chimerax.core.commands import (BoolArg, Bounded, CmdDesc, ColormapArg,
                                    ColormapRangeArg, Int2Arg, IntArg,
                                    SurfacesArg, StringArg, FloatArg, SurfaceArg, AxisArg)
from chimerax.core.commands.cli import EnumOf
from chimerax.map.volumecommand import volume
from chimerax.std_commands.cd import (cd)
from os.path import exists
import numpy
from chimerax.surface.dust import largest_blobs_triangle_mask 
from numpy import (arccos, array, full, inf, isnan, mean, nan, nanmax, nanmean,
                   nanmin, pi, ravel_multi_index, sign, split, sqrt, subtract,
                   count_nonzero, swapaxes, savetxt, column_stack,nansum, nanstd,
                   unique, column_stack, round_, int64, abs, digitize, linspace,
                   zeros, where, delete, shape, ravel, min, shape, isin,flip)
from scipy.ndimage import (binary_dilation, binary_erosion,
                           generate_binary_structure, iterate_structure, gaussian_filter, gaussian_laplace)
from scipy.spatial import KDTree
from skimage.morphology import (skeletonize,label)
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from chimerax.map_data import ArrayGridData 
from chimerax.map import volume_from_grid_data
from skimage.feature import canny


In [ ]:
surface=session.models[1]
mask_vol = surface.full_matrix().copy()
mean = numpy.nanmean(mask_vol,where=(mask_vol>110))
dev = numpy.nanstd(mask_vol,where=(mask_vol>110))

masked_image= (mask_vol >= (mean-(dev*.1)))
masking= (masked_image<1)

mask=ArrayGridData(masked_image)
volume_from_grid_data(mask,session)

mask=ArrayGridData(masking)
volume_from_grid_data(mask,session)

rename #3 "masked IMG"
rename #4 "Mask Inverted"
view matrix camera -1,-0.0019373,0,42.868,0.0019373,-1,0,39.424,0,0,1,141.36;
volume #1.1 style image region 0,0,156,834,767,156 step 1 showOutlineBox true;
volume #3 style image region 0,0,156,834,767,156 step 1 showOutlineBox true;
volume #4 style image region 0,0,156,834,767,156 step 1 showOutlineBox true;
volume #1,3,4 style image region 0,0,36,834,767,36 voxelsize .1028;

hide #!2,3,4;

save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panela-surface.png;
show #!3 models;
hide #!1 models;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelb-surface.png;
hide #!3 models;
show #!2,4 models;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelc-surface.png;

In [ ]:

blank=zeros(shape(masking))


for n in range(shape(masked_image)[0]):
    edge = canny(masked_image[n,:,:],sigma=1)
    blank[n,:,:] = edge


d=binary_dilation(blank,iterations=1)

dd=binary_dilation(blank,iterations=2)

g= 1*dd-1*d

p=binary_dilation(g,iterations=3)

shave=masked_image*(p+d)

mask=ArrayGridData(blank)
volume_from_grid_data(mask,session)

mask=ArrayGridData(g)
volume_from_grid_data(mask,session)

mask=ArrayGridData(shave)
volume_from_grid_data(mask,session)

volume #5-7 style image region 0,0,36,834,767,36;
volume #5-7 voxelSize .1028 color #00ff00;
hide #!3-4;


rename #6 expansion
rename #5 Canny2dEdges
rename #7 Shaved

hide #!2-7;
show #!5;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-paneld-surface.png;
hide #!5;
show #!6;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panele-surface.png;
hide #!6;
show #!2,7;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelf-surface.png;
###this is all to generate a mask###

In [ ]:

y_l=gaussian_laplace((mask_vol*((shave+0) - (masking+0))),sigma=.3,truncate=3)
y=gaussian_laplace((mask_vol),sigma=.3,truncate=8)

shadow=y
shadow_thresh=shadow>=(.8*numpy.max(shadow))

shadow_l=y_l
shadow_thresh_l=shadow_l>=(.65*numpy.max(shadow_l))

mask=ArrayGridData(y_l)
volume_from_grid_data(mask,session)

mask=ArrayGridData(y)
volume_from_grid_data(mask,session)

mask=ArrayGridData(shadow_thresh)
volume_from_grid_data(mask,session)

mask=ArrayGridData(shadow_thresh_l)
volume_from_grid_data(mask,session)

volume #8-11 style image region 0,0,36,834,767,36;
volume #8-11 voxelSize .1028 color #00ff00;
hide #!3-7;

rename #8 "ROIs LoG"
rename #9 "Full LoG"
rename #10 "ROIs Threshed Log"
rename #11 "Full Threshed Log"

hide #!2-11;
show #!8;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelg-fullROIsLoG.png;
hide #!8;
show #!9;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelh-LoG.png;
hide #!9;
show #!2,10;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-paneli-ROIsThreshedLog.png;
hide #!2,10;
show #!11;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelj-FullThreshedLog.png;

In [ ]:
img_shave = y*(shadow_thresh+shadow_thresh_l)
dusted_i  =zeros(shape(img_shave))
img_iiii = img_shave >= (0.87*numpy.max(img_shave))

mask=ArrayGridData(img_shave)
volume_from_grid_data(mask,session)

mask=ArrayGridData(img_iiii)
volume_from_grid_data(mask,session)

volume #12-13 style image region 0,0,36,834,767,36;
volume #12-13 voxelSize .1028 color #00ff00;
hide #!3-11;

rename #12 "ImageWithShavedEdgesByLog"
rename #13 "Voids_threshimg"


hide #!2-13;
show #!2,12;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelk-ImageWithShavedEdgesByLog.png;
hide #!2,12;
show #!13;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panell-Voids_threshImg.png;

In [ ]:
heightMax = numpy.max(where(shave<=0.82),axis=1)[0]
heightLimit= heightMax * 0.95
dusted_iiii  =zeros(shape(img_shave))
img_int=img_iiii.astype("uint8")
for n in range(shape(masked_image)[0]):
    void = label(img_int[n,:,:],background=0)
    RegionID, RegionSize= unique(void,return_counts=True)
    exclusion=numpy.asarray(RegionSize>58).nonzero()    
    dusted_values=RegionID[exclusion]
    count=numpy.sum(exclusion)
    if count <= 10 or n>heightLimit:
        dusted_iiii[n,:,:] = numpy.ones(numpy.shape(void))
    else:
        dusted_iiii[n,:,:] = void==dusted_values.any()


In [ ]:
voids_found=((img_iiii-dusted_iiii))>0 
mask=ArrayGridData(voids_found)
volume_from_grid_data(mask,session)

volume #14 style image region 0,0,36,834,767,36;
volume #14 voxelSize .1028 color #00ff00;
hide #!2-14;

rename #14 "voids_1iteration"

show #!1,14;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelm-Voids1iterationWimage.png;

In [ ]:


img_i = img_shave >= (0.79*numpy.max(img_shave))
img_ii = img_shave >= (0.82*numpy.max(img_shave))
img_iii = img_shave >= (0.85*numpy.max(img_shave))
img_iiii = img_shave >= (0.87*numpy.max(img_shave))

dusted_i  =zeros(shape(img_shave))
dusted_ii  =zeros(shape(img_shave))
dusted_iii  =zeros(shape(img_shave))
dusted_iiii  =zeros(shape(img_shave))

In [ ]:
heightMax = numpy.max(where(shave<=0.82),axis=1)[0]
heightLimit= heightMax * 0.95
img_int=img_i.astype("uint8")
for n in range(shape(masked_image)[0]):
    void = label(img_int[n,:,:],background=0)
    RegionID, RegionSize= unique(void,return_counts=True)
    exclusion=numpy.asarray(RegionSize>58).nonzero()    
    dusted_values=RegionID[exclusion]
    count=numpy.sum(exclusion)
    if count <= 10 or n>heightLimit:
        dusted_i[n,:,:] = numpy.ones(numpy.shape(void))
    else:
        dusted_i[n,:,:] = void==dusted_values.any()

heightMax = numpy.max(where(shave<=0.82),axis=1)[0]
heightLimit= heightMax * 0.95
img_int=img_ii.astype("uint8")
for n in range(shape(masked_image)[0]):
    void = label(img_int[n,:,:],background=0)
    RegionID, RegionSize= unique(void,return_counts=True)
    exclusion=numpy.asarray(RegionSize>58).nonzero()    
    dusted_values=RegionID[exclusion]
    count=numpy.sum(exclusion)
    if count <= 10 or n>heightLimit:
        dusted_ii[n,:,:] = numpy.ones(numpy.shape(void))
    else:
        dusted_ii[n,:,:] = void==dusted_values.any()

heightMax = numpy.max(where(shave<=0.82),axis=1)[0]
heightLimit= heightMax * 0.95
img_int=img_iii.astype("uint8")
for n in range(shape(masked_image)[0]):
    void = label(img_int[n,:,:],background=0)
    RegionID, RegionSize= unique(void,return_counts=True)
    exclusion=numpy.asarray(RegionSize>58).nonzero()    
    dusted_values=RegionID[exclusion]
    count=numpy.sum(exclusion)
    if count <= 10 or n>heightLimit:
        dusted_iii[n,:,:] = numpy.ones(numpy.shape(void))
    else:
        dusted_iii[n,:,:] = void==dusted_values.any()

heightMax = numpy.max(where(shave<=0.82),axis=1)[0]
heightLimit= heightMax * 0.95
img_int=img_iiii.astype("uint8")
for n in range(shape(masked_image)[0]):
    void = label(img_int[n,:,:],background=0)
    RegionID, RegionSize= unique(void,return_counts=True)
    exclusion=numpy.asarray(RegionSize>58).nonzero()    
    dusted_values=RegionID[exclusion]
    count=numpy.sum(exclusion)
    if count <= 10 or n>heightLimit:
        dusted_iiii[n,:,:] = numpy.ones(numpy.shape(void))
    else:
        dusted_iiii[n,:,:] = void==dusted_values.any()

In [ ]:
voids_found=((img_i-dusted_i) + (img_ii-dusted_ii) + (img_iii-dusted_iii) + (img_iiii-dusted_iiii))>0      
mask=ArrayGridData((voids_found))
volume_from_grid_data(mask,session)

volume #15 style image region 0,0,36,834,767,36;
volume #15 voxelSize .1028 color #00ff00;
hide #!3-15;

rename #15 "voids_4iteration"

show #!1,15;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-paneln-Voids4iterationWimage.png;
close #3-15

open X:\Macropinocytosis\BMM-KOs\20230823cs5cell1WT\*488nm*.tif;

#surface render#

material dull;

lighting full;

lighting depthcue false;

set bg transparent;

windowsize 800 800;

volume #1 step 3 level 145 color #023b55 voxelsize 0.1028;

view matrix camera -1,-0.0019373,0,42.868,0.0019373,-1,0,39.424,0,0,1,141.36;

scalebar 15 xpos .7 ypos .05;

volume #1,3, region 0,0,0,834,767,217;

volume #3 level 0.85
view matrix camera -0.84986,-0.39637,0.34731,97.695,0.52642,-0.60747,0.59487,116.17,-0.024807,0.68839,0.72492,94.555;
surface dust #3 size 1;



save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelo-Voids4iterationsurfaceFLM_1um-6um.png;
hide #!1;
save C:\Users\LLSM\Documents\Yoseph\Dissertation\ch6-fig2-panelo-Voids4iterationsurface_1um-6um.png;